In [1]:
# import libraries
import pandas as pd
import os
import csv
import mediapipe as mp
import numpy as np 
import cv2 as cv
from sklearn.model_selection import train_test_split,GridSearchCV,StratifiedKFold
import matplotlib.pyplot as plt
import time

from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.metrics import accuracy_score
import pickle

In [2]:
mp_drawing= mp.solutions.drawing_utils # drawing helper
mp_pose= mp.solutions.pose # mediapipe model

In [3]:
def Resize(frame,scale):
            height=int(frame.shape[0]* scale)
            width=int(frame.shape[1]*scale)
            dimensions=(width,height)

            return cv.resize(frame,dimensions,interpolation=cv.INTER_CUBIC)

Detecting pose landmarks

In [4]:
# Get Camera feed

video=cv.VideoCapture(0)

with mp_pose.Pose(model_complexity=2,min_detection_confidence=0.5,min_tracking_confidence=0.5) as pose:
    # set time 
    time1= 0


    while video.isOpened():
        ret, frame= video.read()
        frame.flags.writeable=False
        re_img=Resize(frame,1.5)

        # change color feed
        
        img=cv.cvtColor(re_img,cv.COLOR_BGR2RGB)
        
        img=cv.flip(img,1)
        # make detections
        results=pose.process(img)

        time2= time.time()

        if (time2 - time1)>0:
            
            fps=round(1.0/(time2-time1),1)

            cv.putText(img,f'FPS : {fps}',(10,30),cv.FONT_HERSHEY_PLAIN,2,(0,255,0),2)
        

        time1=time2
        img.flags.writeable=True
        img =cv.cvtColor(img,cv.COLOR_RGB2BGR)

        # Pose Detections
        mp_drawing.draw_landmarks(img, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,mp_drawing.DrawingSpec(color=(178, 247, 31),thickness=2,circle_radius=1),mp_drawing.DrawingSpec(color=(129, 102, 108),thickness=1))

        
        cv.imshow('Raw Webcam Feed', img)

        if cv.waitKey(1) & 0xFF == ord('q'):
            break

video.release()
cv.destroyAllWindows()

Genearating Dataset for yoga poses consisting 4 classes - Downdog,goddess,plank,Tree,Warrior2

In [5]:
num_objects=len(results.pose_landmarks.landmark)
yoga=['class']
yoga += list((np.array([[f'x_{i}',f'y_{i}',f'z_{i}',f'v_{i}'] for i in range(1,num_objects+1)]).flatten()))
with open('landmarks.csv','w',newline='') as f:
    file=csv.writer(f,delimiter=',')
    file.writerow(yoga)


In [23]:
Pose_name=os.listdir(r'C:\Users\adm88\ML_deployment\Object_detection\yoga_pose_media_pipe\TEST')

with mp_pose.Pose(model_complexity=2,static_image_mode=True,min_detection_confidence=0.5,min_tracking_confidence=0.5) as pose:

    for i in Pose_name:
        class_name=i
        img_dir_path=r'C:\Users\adm88\ML_deployment\Object_detection\yoga_pose_media_pipe\TEST\{0}'.format(i)
        for j in os.listdir(img_dir_path):
            pic=cv.imread(img_dir_path+"\\"+j)

            # detecting poses
            
            pic=cv.cvtColor(pic,cv.COLOR_BGR2RGB)
            pic=cv.resize(pic,(540,540),cv.INTER_LINEAR)
        
            
            # make detections
            result=pose.process(pic)
             # change color feed
        
            pic=cv.cvtColor(pic,cv.COLOR_RGB2BGR)
            

            # Pose Detections
            
            mp_drawing.draw_landmarks(pic, result.pose_landmarks, mp_pose.POSE_CONNECTIONS,mp_drawing.DrawingSpec(color=(178, 247, 31),thickness=2,circle_radius=1),mp_drawing.DrawingSpec(color=(129, 102, 108),thickness=1))
            
            Class_landmarks= result.pose_landmarks.landmark
            landmarks= list(np.array([[i.x,i.y,i.x,i.visibility] for i in Class_landmarks]).flatten())
            landmarks.insert(0,class_name)

            with open('landmarks.csv','a',newline='') as f:
                File= csv.writer(f, delimiter=',')
                File.writerow(landmarks)
                
            
            #cv.imshow('Raw Feed', pic)

            cv.waitKey(1) 
            
            cv.destroyAllWindows()

Adding custom data with webcam



In [4]:
class_name= 'Normal'
cap=cv.VideoCapture(0)
with mp_pose.Pose(model_complexity=2,static_image_mode=True,min_detection_confidence=0.5,min_tracking_confidence=0.5) as pose:

    while cap.isOpened():
        _, frame=cap.read()
        frame.flags.writeable=False

            # detecting poses
        
        pic=cv.cvtColor(frame,cv.COLOR_BGR2RGB)
        pic=cv.resize(pic,(540,540),cv.INTER_LINEAR)
        
            
            # make detections
        result=pose.process(pic)
             # change color feed
        pic.flags.writeable=True
        pic=cv.cvtColor(pic,cv.COLOR_RGB2BGR)
            

            # Pose Detections
            
        mp_drawing.draw_landmarks(pic, result.pose_landmarks, mp_pose.POSE_CONNECTIONS,mp_drawing.DrawingSpec(color=(178, 247, 31),thickness=2,circle_radius=1),mp_drawing.DrawingSpec(color=(129, 102, 108),thickness=1))
            
        Class_landmarks= result.pose_landmarks.landmark
        landmarks= list(np.array([[i.x,i.y,i.x,i.visibility] for i in Class_landmarks]).flatten())
        landmarks.insert(0,class_name)

        with open('landmarks.csv','a',newline='') as f:
            File= csv.writer(f, delimiter=',')
            File.writerow(landmarks)
                
            
            cv.imshow('Raw Feed', pic)

        cv.waitKey(1) 
cap.release()          
cv.destroyAllWindows()

KeyboardInterrupt: 

In [8]:
results.pose_landmarks.landmark[0]

x: 0.5981493592262268
y: 0.38991042971611023
z: -1.320794939994812
visibility: 0.9997200965881348

In [5]:
df=pd.read_csv(r'C:\Users\adm88\ML_deployment\Object_detection\yoga_pose_media_pipe\landmarks.csv')
df.head()

,class,x_1,y_1,z_1,v_1,x_2,y_2,z_2,v_2,x_3,...,z_31,v_31,x_32,y_32,z_32,v_32,x_33,y_33,z_33,v_33
0,downdog,0.524543,0.739313,0.524543,0.999333,0.501234,0.754761,0.501234,0.999382,0.497772,...,0.884256,0.006587,0.791916,0.918564,0.791916,0.279184,0.784176,0.909822,0.784176,0.005242
1,downdog,0.550666,0.762332,0.550666,0.999724,0.572692,0.773962,0.572692,0.999440,0.575015,...,0.143413,0.925378,0.259711,0.889642,0.259711,0.151637,0.249647,0.895167,0.249647,0.940103
2,downdog,0.591179,0.718564,0.591179,0.998485,0.614638,0.725264,0.614638,0.993898,0.619974,...,0.055977,0.966626,0.215033,0.876577,0.215033,0.239551,0.201332,0.898214,0.201332,0.967450
3,downdog,0.527703,0.742895,0.527703,0.998805,0.545204,0.766965,0.545204,0.995739,0.549692,...,0.160866,0.504536,0.276530,0.871081,0.276530,0.040829,0.272185,0.882554,0.272185,0.535279
4,downdog,0.383210,0.672364,0.383210,0.998244,0.357112,0.693990,0.357112,0.998268,0.352921,...,0.852095,0.003518,0.735646,0.910208,0.735646,0.087242,0.727957,0.897249,0.727957,0.002657


In [6]:
label=LabelEncoder()

le=label.fit_transform(df['class'].values)
label.classes_

array(['Normal', 'downdog', 'goddess', 'plank', 'warrior2'], dtype=object)

In [7]:
def Split(data,test_size,random_state):
    x=data.drop(columns='class')
    y=le

    x_train,x_test,y_train,y_test= train_test_split(x,y,test_size=test_size,random_state=random_state)
    return x_train,x_test,y_train,y_test

x_train,x_test,y_train,y_test=Split(df,0.15,123)

In [8]:
def Scaling( x_train,x_test):
    scale= StandardScaler()
    x_train_scaled= scale.fit_transform(x_train)
    x_test_scaled= scale.fit_transform(x_test)

    
    return x_train_scaled,x_test_scaled 

x_train_scaled, x_test_scaled = Scaling(x_train,x_test)

In [9]:
xgb=XGBClassifier(tree_method='gpu_hist').fit(x_train,y_train)

score1=xgb.score(x_train,y_train)
score1


1.0

In [10]:
predictions=xgb.predict(x_test)
predictions

array([4, 2, 1, 4, 3, 3, 0, 0, 1, 2, 0, 1, 0, 0, 0, 3, 1, 1, 1, 3, 4, 4,
       0, 0, 1, 4, 0, 0, 1, 0, 3, 0, 2, 3, 1, 1, 2, 3, 1, 4, 0, 0, 0, 2,
       2, 3, 4, 0, 4, 3, 0, 3, 2, 3, 0, 4, 4, 0, 3, 1, 3, 4, 0, 0, 1, 0,
       3, 2, 4, 1, 3, 4, 3, 2, 2, 1, 1, 4, 1, 1, 4, 4, 0, 0], dtype=int64)

In [11]:
label.inverse_transform(predictions)

array(['warrior2', 'goddess', 'downdog', 'warrior2', 'plank', 'plank',
       'Normal', 'Normal', 'downdog', 'goddess', 'Normal', 'downdog',
       'Normal', 'Normal', 'Normal', 'plank', 'downdog', 'downdog',
       'downdog', 'plank', 'warrior2', 'warrior2', 'Normal', 'Normal',
       'downdog', 'warrior2', 'Normal', 'Normal', 'downdog', 'Normal',
       'plank', 'Normal', 'goddess', 'plank', 'downdog', 'downdog',
       'goddess', 'plank', 'downdog', 'warrior2', 'Normal', 'Normal',
       'Normal', 'goddess', 'goddess', 'plank', 'warrior2', 'Normal',
       'warrior2', 'plank', 'Normal', 'plank', 'goddess', 'plank',
       'Normal', 'warrior2', 'warrior2', 'Normal', 'plank', 'downdog',
       'plank', 'warrior2', 'Normal', 'Normal', 'downdog', 'Normal',
       'plank', 'goddess', 'warrior2', 'downdog', 'plank', 'warrior2',
       'plank', 'goddess', 'goddess', 'downdog', 'downdog', 'warrior2',
       'downdog', 'downdog', 'warrior2', 'warrior2', 'Normal', 'Normal'],
      dtype=objec

In [12]:
with open('Yoga_ML.pkl','wb') as f:
    pickle.dump(xgb,f)

In [2]:
# Get Camera feed
with open('Yoga_ML.pkl','rb') as f:
    model_gbc= pickle.load(f)
video=cv.VideoCapture(0)

with mp_pose.Pose(model_complexity=2,min_detection_confidence=0.5,min_tracking_confidence=0.5) as pose:
    # set time 
    time1= 0


    while video.isOpened():
        ret, frame= video.read()
        frame.flags.writeable=False
        

        # change color feed
        
        img=cv.cvtColor(frame,cv.COLOR_BGR2RGB)
        
        img=cv.flip(img,1)
        # make detections
        result=pose.process(img)

        time2= time.time()

        if (time2 - time1)>0:
            
            fps=round(1.0/(time2-time1),1)

            cv.putText(img,f'FPS : {fps}',(10,30),cv.FONT_HERSHEY_PLAIN,2,(0,255,0),2)
        

        time1=time2
        img.flags.writeable=True
        img =cv.cvtColor(img,cv.COLOR_RGB2BGR)

        # Pose Detections
        Class_landmarks= result.pose_landmarks.landmark
        landmarks= list(np.array([[i.x,i.y,i.x,i.visibility] for i in Class_landmarks]).flatten())
        mp_drawing.draw_landmarks(img, result.pose_landmarks, mp_pose.POSE_CONNECTIONS,mp_drawing.DrawingSpec(color=(178, 247, 31),thickness=2,circle_radius=1),mp_drawing.DrawingSpec(color=(129, 102, 108),thickness=1))
        A=pd.DataFrame([landmarks])
    
        scaled_A=StandardScaler().fit_transform(A)
        Face_body_class= model_gbc.predict(A)
        texts=label.inverse_transform(Face_body_class)[0]
        prob_class=model_gbc.predict_proba(A)[0]
        print(texts,prob_class)


        cv.rectangle(img,(20,100),(40,200),(0, 191, 255),-1)
        cv.putText(img,texts,(200,30),cv.FONT_HERSHEY_PLAIN, 2,(255, 0, 255),2,cv.LINE_AA)
        
        cv.imshow('Raw Webcam Feed', img)



        if cv.waitKey(1) & 0xFF == ord('q'):
            break

video.release()
cv.destroyAllWindows()

NameError: name 'pickle' is not defined